**Import dependicies**




In [1]:
%matplotlib nbagg

import numpy as np
import numpy.linalg as lin

from IPython.display import display,HTML,clear_output
import time

import cv2

import matplotlib as mpl
import matplotlib.pyplot as plt

import math
from math import pi as PI

mpl.use('nbagg')

from matplotlib import animation
mpl.rc('animation', html='html5') #display animated plots inline

from robmob.robot import Robot
from robmob.sensors import KinectRGBSensor
from robmob.sensors import KinectDepthSensor
from robmob.visualization import Visualizer


**Connect to Robot**

In [2]:
ip_robot = '192.168.0.109'
robot = Robot(ip_robot)
robot.connect()

In [3]:
kinect = KinectRGBSensor()
robot.add_sensor(kinect)
depth_sensor = KinectDepthSensor()
robot.add_sensor(depth_sensor)

qrTag_1 = cv2.imread('qr2.png', 0)          # trainImage1
qrTag_2 = cv2.imread('qr.jpg', 0)          # trainImage2
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT


In [4]:
# -------------------------------------------------
# | input parameter  |-> trainImage
#                    |-> queryImage
# -------------------------------------------------
# | output parameter |-> (numberOfDMatch, listOfDMatchPoints) 
# -------------------------------------------------
def get_inlines(queryImg, tag):
    queryImage = np.array(queryImg)
    MIN_MATCH_COUNT = 1

    kp1, des1 = sift.detectAndCompute(queryImage,None)    
    kp2, des2 = sift.detectAndCompute(tag,None)     
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.5*n.distance:
            good.append(m)

    if len(good) > MIN_MATCH_COUNT:
        return len(good), good
    else:
        return 0, 0

In [5]:
# -------------------------------------------------
# | input parameter  |-> tag
#                    |-> angle
#                    |-> speed
# -------------------------------------------------
# | output parameter |-> "the orientation of the robot must be toward the tag"
# -------------------------------------------------
def scan_for_max_inlines(tag, angle, speed):
    max_inlines = False
    inlines = 0
    val=[]
    while not max_inlines:
        robot.angular_movement(angle, speed)
        image = kinect.peek_data()
        time.sleep(1)
        new_inlines = get_inlines(image, tag)[0]
#         print("new_inlines " , new_inlines, "inlines: ", inlines)
        kinect.buffer.clear()
        
        if inlines <= new_inlines:
            inlines = new_inlines
#             print("not max_inlines")
        else:
            max_inlines = False
#             print("yes")
            return

In [6]:
# -------------------------------------------------
# | input parameter  |-> list of DMatches (from SIFT algorithm)
#                    |-> image where to find coords (queryImage)
#                    |-> QR tag                     (trainImage)
# -------------------------------------------------
# | output parameter |-> list of coordonates
# -------------------------------------------------
def find_coords(dMatchList, queryImg, tag):
    queryImage = np.array(queryImg)
    
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(queryImage, None)
    kp2, des2 = sift.detectAndCompute(tag, None)
    
    points = []
    for dmatch in dMatchList:
        coords = kp1[dmatch.queryIdx].pt
        points.append(coords)
    return points

In [7]:
# -------------------------------------------------
# | input parameter  |-> Depth data
#                    |-> list of points (coordinates)
# -------------------------------------------------
# | output parameter |-> distance (z) to tag
# -------------------------------------------------
def get_z_from_listOfPoints(depth_data, points):
    zInEachPoint = []
    for point in points:
        z = depth_data[(point[0], point[1])]
        if z > 0 and z < 3 :
            zInEachPoint.append(z)
     
    print('zInEachPoint', zInEachPoint) 
    avg = sum(zInEachPoint)/len(zInEachPoint)
    
    zInEachPointWithoutAberrant = []
    for z in zInEachPoint:
        if ((avg*0.8) < z < (avg*1.2)):
            zInEachPointWithoutAberrant.append(z)
    
    
    print(zInEachPointWithoutAberrant)
    return sum(zInEachPointWithoutAberrant)/len(zInEachPointWithoutAberrant)
    

In [9]:
# ***  MAIN  ***

for tag in [qrTag_2, qrTag_2]:
    time.sleep(1)
    print('Let''s Do IT')
    time.sleep(1)
    scan_for_max_inlines(tag, PI/8,1)
    scan_for_max_inlines(tag, -PI/16,0.5)
    time.sleep(1)
    queryImage = kinect.peek_data()
    time.sleep(1)
    dMatchList = get_inlines(queryImage, tag)[1]
    points = find_coords(dMatchList, queryImage, tag)
    time.sleep(1)
    depth_data = depth_sensor.peek_data()
    d = get_z_from_listOfPoints(depth_data, points)
    time.sleep(1)
    robot.linear_movement_precise(d*0.8, 0.1)
